## create model 

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
import keras.backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement=True
session = tf.Session(config=config)
K.set_session(session)

Using TensorFlow backend.


In [3]:
# from tensorflow.python import debug as tf_debug
# K.set_session(
#     tf_debug.TensorBoardDebugWrapperSession(tf.Session(), "DESKTOP-NAQHFD5:6006"))

In [4]:
from model.yolov3 import YOLO_V3
import config 

In [5]:
input_shape = (416,416,3)
with tf.device('/cpu:0'):
    yolo = YOLO_V3(input_shape,config.num_classes)
    yolo_body,model = yolo.create_yolo()

yolo_body.load_weights(r'E:\yolov3\model_data\yolo_416.h5', by_name=True,skip_mismatch=True)
for i in range(len(model.layers)-7):  
    model.layers[i].trainable = False

feats_shape: (None, None, None, 3, 10) y_true_shape: (None, None, None, 3, 14)
gt_t_xy_shape: (None, None, None, 3, 2) pred_t_wh_shape: (None, None, None, 3, 2)
gt_box_shape: (None, None, None, 3, 4) pred_box_shape: (None, None, None, 3, 1, 4) object_mask_shape: (None, None, None, 3, 1)
s_iou: (None, None, 3, None) s_best_iou (None, None, 3) s_gt_box_shape (None, 4)
ignore_mask_shape: (None, None, None, 3, 1) object_mask_shape (None, None, None, 3, 1)
loss_xy_shape: (None, None, None, 3, 2)
feats_shape: (None, None, None, 3, 10) y_true_shape: (None, None, None, 3, 14)
gt_t_xy_shape: (None, None, None, 3, 2) pred_t_wh_shape: (None, None, None, 3, 2)
gt_box_shape: (None, None, None, 3, 4) pred_box_shape: (None, None, None, 3, 1, 4) object_mask_shape: (None, None, None, 3, 1)
s_iou: (None, None, 3, None) s_best_iou (None, None, 3) s_gt_box_shape (None, 4)
ignore_mask_shape: (None, None, None, 3, 1) object_mask_shape (None, None, None, 3, 1)
loss_xy_shape: (None, None, None, 3, 2)
feats_sh

C:\Program Files\Anaconda3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 30) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
C:\Program Files\Anaconda3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((30,) vs (255,)).
  weight_values[i].shape))
C:\Program Files\Anaconda3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 30) vs (255, 512, 1, 1)).
  weight_values[i].shape))
C:\Program Files\Anaconda3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((30,) vs (255,)).
  weight_values[i].shape))
C:\Program Files\Anaconda3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_75

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 416, 416, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 416, 416, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 416, 416, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [7]:
for i in range(len(model.layers)):  
    print(model.layers[i].name,model.layers[i].trainable)

input_1 False
conv2d_1 False
batch_normalization_1 False
leaky_re_lu_1 False
zero_padding2d_1 False
conv2d_2 False
batch_normalization_2 False
leaky_re_lu_2 False
conv2d_3 False
batch_normalization_3 False
leaky_re_lu_3 False
conv2d_4 False
batch_normalization_4 False
leaky_re_lu_4 False
add_1 False
zero_padding2d_2 False
conv2d_5 False
batch_normalization_5 False
leaky_re_lu_5 False
conv2d_6 False
batch_normalization_6 False
leaky_re_lu_6 False
conv2d_7 False
batch_normalization_7 False
leaky_re_lu_7 False
add_2 False
conv2d_8 False
batch_normalization_8 False
leaky_re_lu_8 False
conv2d_9 False
batch_normalization_9 False
leaky_re_lu_9 False
add_3 False
zero_padding2d_3 False
conv2d_10 False
batch_normalization_10 False
leaky_re_lu_10 False
conv2d_11 False
batch_normalization_11 False
leaky_re_lu_11 False
conv2d_12 False
batch_normalization_12 False
leaky_re_lu_12 False
add_4 False
conv2d_13 False
batch_normalization_13 False
leaky_re_lu_13 False
conv2d_14 False
batch_normalization_14

In [8]:
from keras.optimizers import Adam

## create generator

In [9]:
from model.generator import Generator
from keras.callbacks import TensorBoard

In [10]:
tensorboard = TensorBoard(r'E:\yolov3\tf')

In [11]:
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(model)
# complie
parallel_model.compile(optimizer=Adam(lr=1e-3), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: K.mean(y_pred)})

feats_shape: (None, None, None, 3, 10) y_true_shape: (None, None, None, 3, 14)
gt_t_xy_shape: (None, None, None, 3, 2) pred_t_wh_shape: (None, None, None, 3, 2)
gt_box_shape: (None, None, None, 3, 4) pred_box_shape: (None, None, None, 3, 1, 4) object_mask_shape: (None, None, None, 3, 1)
s_iou: (None, None, 3, None) s_best_iou (None, None, 3) s_gt_box_shape (None, 4)
ignore_mask_shape: (None, None, None, 3, 1) object_mask_shape (None, None, None, 3, 1)
loss_xy_shape: (None, None, None, 3, 2)
feats_shape: (None, None, None, 3, 10) y_true_shape: (None, None, None, 3, 14)
gt_t_xy_shape: (None, None, None, 3, 2) pred_t_wh_shape: (None, None, None, 3, 2)
gt_box_shape: (None, None, None, 3, 4) pred_box_shape: (None, None, None, 3, 1, 4) object_mask_shape: (None, None, None, 3, 1)
s_iou: (None, None, 3, None) s_best_iou (None, None, 3) s_gt_box_shape (None, 4)
ignore_mask_shape: (None, None, None, 3, 1) object_mask_shape (None, None, None, 3, 1)
loss_xy_shape: (None, None, None, 3, 2)
feats_sh

In [12]:
parallel_model.summary()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
lambda_1 (

In [13]:
batch_size = 24
gen = Generator(r'C:\dataset\jinnan2_round1_train_20190305\jinnan2_round1_train_20190305\restricted\train.txt',
                batch_size = batch_size,
                num_classes = config.num_classes,
                shape  = (416,416))

In [14]:

res = parallel_model.fit_generator(gen,
                          steps_per_epoch =gen.num_samples()// batch_size,
                          epochs = 50,
                          max_queue_size=128,
                          workers=4,
                          verbose=1)

Epoch 1/50
40/40 [==============================] - 30s 747ms/step - loss: 2900.6628
Epoch 2/50
40/40 [==============================] - 15s 364ms/step - loss: 280.7727
Epoch 3/50
40/40 [==============================] - 15s 366ms/step - loss: 170.6424
Epoch 4/50
40/40 [==============================] - 14s 361ms/step - loss: 129.0811
Epoch 5/50
40/40 [==============================] - 15s 363ms/step - loss: 105.8816
Epoch 6/50
40/40 [==============================] - 15s 364ms/step - loss: 91.0800
Epoch 7/50
40/40 [==============================] - 15s 379ms/step - loss: 81.3749
Epoch 8/50
40/40 [==============================] - 15s 374ms/step - loss: 75.4453
Epoch 9/50
40/40 [==============================] - 15s 368ms/step - loss: 70.2651
Epoch 10/50
40/40 [==============================] - 15s 366ms/step - loss: 66.3963
Epoch 11/50
40/40 [==============================] - 15s 372ms/step - loss: 63.0323
Epoch 12/50
40/40 [==============================] - 14s 362ms/step - loss: 61.

In [17]:

for layer in model.layers:  
    layer.trainable = True
    
parallel_model.compile(optimizer=Adam(lr=1e-4), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: K.mean(y_pred)})
parallel_model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
lambda_1 (

In [18]:
res = parallel_model.fit_generator(gen,
                          steps_per_epoch =gen.num_samples()// batch_size,
                          validation_data=gen,
                          validation_steps = 5,
                          epochs = 1000,
                          max_queue_size=32,
                          workers=4,
                          verbose=1)

Epoch 1/1000
40/40 [==============================] - 84s 2s/step - loss: 28.6065 - val_loss: 31.5371
Epoch 2/1000
40/40 [==============================] - 41s 1s/step - loss: 17.5985 - val_loss: 25.4315
Epoch 3/1000
40/40 [==============================] - 40s 1s/step - loss: 12.2033 - val_loss: 23.0284
Epoch 4/1000
40/40 [==============================] - 40s 1s/step - loss: 8.4960 - val_loss: 20.1807
Epoch 5/1000
40/40 [==============================] - 40s 1s/step - loss: 5.7783 - val_loss: 19.0661
Epoch 6/1000
40/40 [==============================] - 41s 1s/step - loss: 4.6889 - val_loss: 20.1893
Epoch 7/1000
40/40 [==============================] - 41s 1s/step - loss: 3.7329 - val_loss: 19.6718
Epoch 8/1000
40/40 [==============================] - 41s 1s/step - loss: 3.0254 - val_loss: 18.5544
Epoch 9/1000
40/40 [==============================] - 41s 1s/step - loss: 2.3674 - val_loss: 16.9748
Epoch 10/1000
40/40 [==============================] - 41s 1s/step - loss: 2.0981 - val_

KeyboardInterrupt: 

In [19]:
model.save_weights(r'stage3.hdf5')